In [3]:
import os
import pandas as pd
import numpy as np

DIR = "./data/Raw/logRaws/"

NUM_SAMPLES = 2662
NUM_FEATURES = 6
NUM_FRAMES = 1000

file_names = os.listdir(DIR)
input_array = np.empty((NUM_SAMPLES, NUM_FRAMES, NUM_FEATURES))
output_array = np.empty((NUM_SAMPLES, NUM_FRAMES, 1))

# prepare data
def read_files(DIR):
    all_files = os.listdir(DIR)

    input_data = np.empty((0, 6), float)
    output_data = np.empty((0, 1), int)

    for file in all_files:
        file_path = os.path.join(DIR, file)
        df = pd.read_csv(file_path, usecols=[1, 3, 19, 20, 21, 25, 26, 27], header=None)
        if df.shape[0] < 999:
            continue

        df.columns = ['frame', 'status', 'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
        df_input = df[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
        df_input = df_input.astype('float32')
        df_output = df[['status']]
        df_output = df_output.astype('int')
        df_frames = df[['frame']]
        df_frames = df_frames.astype('int')

        input_data = np.append(input_data, df_input, axis=0)
        output_data = np.append(output_data, df_output, axis=0)

    input_data = input_data.reshape(NUM_SAMPLES, NUM_FRAMES, NUM_FEATURES)
    output_data = output_data.reshape(NUM_SAMPLES, NUM_FRAMES, 1)

    return input_data, output_data

input_data, output_data = read_files(DIR)

print(input_data.shape)
print(output_data.shape)

(2662, 1000, 6)
(2662, 1000, 1)


In [4]:
from keras.utils import to_categorical

# one-hot encoding
output_data = to_categorical(output_data)

# split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=42)

# build model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential([
    LSTM(64, input_shape=(NUM_FRAMES, NUM_FEATURES), return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    Dense(5, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


# evaluate model
score = model.evaluate(X_test, y_test, batch_size=32)
print(score)

# save model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1000, 64)          18176     
                                                                 
 dropout (Dropout)           (None, 1000, 64)          0         
                                                                 
 lstm_1 (LSTM)               (None, 1000, 64)          33024     
                                                                 
 dropout_1 (Dropout)         (None, 1000, 64)          0         
                                                                 
 dense (Dense)               (None, 1000, 5)           325       
                                                                 
Total params: 51525 (201.27 KB)
Trainable params: 51525 (201.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
67/67 [

In [5]:
# print one sample: the frame number when the state changes
def print_frame_change(data):
    for i in range(1, data.shape[0]):
        if data[i] != data[i-1]:
            print(i)

# print this for predicted and actual
print(model.predict(X_test)[1])
print(y_test[1])


17/17 [==============================] - 6s 243ms/step
[[5.6077075e-01 2.5765046e-01 2.8066266e-01 1.5480821e-01 2.2460966e-01]
 [6.4479434e-01 9.4652124e-02 1.5543601e-01 2.4410050e-02 7.6001674e-02]
 [7.8278685e-01 2.7747856e-02 8.0124587e-02 4.1512419e-03 2.8632352e-02]
 ...
 [9.9694765e-01 3.2675972e-03 3.9347599e-04 5.1415784e-05 7.8103592e-05]
 [9.9699897e-01 3.3009190e-03 3.6535921e-04 4.8993556e-05 7.7777084e-05]
 [9.9706650e-01 3.3368755e-03 3.3790612e-04 4.6797672e-05 7.6881290e-05]]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
